In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC

In [2]:
upstream = ['section-01-get-data', 'section-04-autosklearn-auto-ml']
product = None

In [3]:
# Parameters
upstream = {
    "section-04-autosklearn-auto-ml": {
        "categorical_columns": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-04-autosklearn-auto-ml-categorical_columns.pkl",
        "X_train": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-04-autosklearn-auto-ml-X_train.pkl",
        "X": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-04-autosklearn-auto-ml-X.pkl",
        "y_train": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-04-autosklearn-auto-ml-y_train.pkl",
        "y": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-04-autosklearn-auto-ml-y.pkl",
        "nb": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-04-autosklearn-auto-ml.ipynb",
    },
    "section-01-get-data": {
        "train": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-01-get-data-train.pkl",
        "df": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-01-get-data-df.pkl",
        "nb": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-01-get-data.ipynb",
    },
}
product = {
    "X_val": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-05-data-preparation-X_val.pkl",
    "X_train": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-05-data-preparation-X_train.pkl",
    "y_train": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-05-data-preparation-y_train.pkl",
    "y_val": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-05-data-preparation-y_val.pkl",
    "nb": "/content/Reproducible-Data-Analysis-With-Jupyter-Notebooks-Standard-Bank-Virtual-Internship/output/section-05-data-preparation.ipynb",
}


In [4]:
train = pickle.loads(Path(upstream['section-01-get-data']['train']).read_bytes())
X = pickle.loads(Path(upstream['section-04-autosklearn-auto-ml']['X']).read_bytes())
categorical_columns = pickle.loads(Path(upstream['section-04-autosklearn-auto-ml']['categorical_columns']).read_bytes())
y = pickle.loads(Path(upstream['section-04-autosklearn-auto-ml']['y']).read_bytes())

## 05) Data Preparation

In [5]:
## One hot encode the categorical features
##Leave all categories to represent missing values 
X_oh_encoded = pd.concat([X, pd.get_dummies(X[categorical_columns])], axis=1)
X_oh_encoded = X_oh_encoded.drop(columns=categorical_columns)

## fill the missing numerical columns
X_oh_encoded['LoanAmount'].fillna(X_oh_encoded['LoanAmount'].mean(), inplace=True)
X_oh_encoded['Loan_Amount_Term'].fillna(X_oh_encoded['Loan_Amount_Term'].median(), inplace=True)
X_oh_encoded['Credit_History'].fillna(X_oh_encoded['Credit_History'].mode()[0], inplace=True)
X_oh_encoded.isna().sum()

ApplicantIncome            0
CoapplicantIncome          0
LoanAmount                 0
Loan_Amount_Term           0
Credit_History             0
Gender_Female              0
Gender_Male                0
Married_No                 0
Married_Yes                0
Dependents_0               0
Dependents_1               0
Dependents_2               0
Dependents_3+              0
Education_Graduate         0
Education_Not Graduate     0
Self_Employed_No           0
Self_Employed_Yes          0
Property_Area_Rural        0
Property_Area_Semiurban    0
Property_Area_Urban        0
dtype: int64

In [6]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(columns=['Loan_ID', 'Loan_Status']), train['Loan_Status'])

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_oh_encoded, y)

In [8]:
svm = SVC()
svm.fit(X_train, y_train)
#print("Accuracy on the Train Dataset : ", accuracy_score(xgb.predict(X_train), y_train))
#print("Accuracy on the Train Dataset : ", accuracy_score(xgb.predict(X_val), y_val))
#print("\nConfusion matrix : \n", confusion_matrix(xgb.predict(X_val), y_val))

SVC()

In [9]:
Path(product['X_train']).parent.mkdir(exist_ok=True, parents=True)
Path(product['X_train']).write_bytes(pickle.dumps(X_train))

Path(product['X_val']).parent.mkdir(exist_ok=True, parents=True)
Path(product['X_val']).write_bytes(pickle.dumps(X_val))

Path(product['y_train']).parent.mkdir(exist_ok=True, parents=True)
Path(product['y_train']).write_bytes(pickle.dumps(y_train))

Path(product['y_val']).parent.mkdir(exist_ok=True, parents=True)
Path(product['y_val']).write_bytes(pickle.dumps(y_val))

4384